说明：
1. 文件格式：ipynb
2. 运行环境：python3.10.2
3. 引入第三方库：pandas、sklearn、matplotlib

项目:用决策树模型进行预测
***
* [一 .数据预处理](#一.数据预处理)
   * [1.1 数据读取](#1.1-数据读取)
   * [1.2 正则提取](#1.2-正则提取)
   * [1.3 缺失值处理](#1.3-缺失值处理)
   * [1.4 数据类型转换](#1.4-数据类型转换)
   * [1.5 描述性统计](#1.5-描述性统计)
   * [1.5 数据转换](#1.4-数据转换)
   * [1.6 异常值检测](#1.5-异常值检测)

***
* [二.建立决策树模型](#二.建立决策树模型)
   * [2.1 划分训练集和测试集](#2.1-划分训练集和测试集)
   * [2.2 建立分类树](#2.2-建立分类树)
   * [2.3 参数调优](#2.3-参数调优)

***
* [三.模型评价](#三.模型评价)
   * [3.1 K折交叉验证](#3.1-K折交叉验证)
   * [3.2 ROC曲线](#3.2-ROC曲线)
   * [3.3 混淆矩阵](#3.3-混淆矩阵)
   

### 一.数据预处理

#### 1.1 数据读取

In [ ]:
import pandas as pd 
# 定义路径，便于数据读取和保存
path = "/Users/guojing/Desktop/"
# 导入数据
df = pd.read_excel(path+"house.xlsx",sheet_name=0)

In [ ]:
# 观察数据结构
df.head()

In [ ]:
#观察第一行、第一列的数据结构，便于正则提取
df.iloc[0,0]

#### 1.2正则提取
说明： 在观察数据结构后，提取以下信息：
1. 在第一列中提取卧室数量、客厅数量、建筑面积、层高、朝向、建造年份 四类数据
2. 在第三列提取折扣信息
3. 在第四列提取价格信息
4. 不提取第五列的每平方米单价信息：原因：信息重复：建筑面积*每平方米单价=价格

In [ ]:
#1.运用了正则表达式提取数据
#2.expend = False ,说明提取格式为Series不是DataFrame：原因：便于后续创建新的DataFrame
re1 = df.all_infro
bedroom = re1.str.extract(r'(\d+(?=室))',expand=False)
livingRoom = re1.str.extract(r'(\d(?=厅))',expand=False)
storey = re1.str.extract(r'(\w层)',expand=False)
area = re1.str.extract(r'(\d+\.?\d+(?=㎡))',expand=False)
orientation = re1.str.extract(r'(\w+向)',expand=False)
constructionTime = re1.str.extract(r'(\d+(?=年))',expand=False)
discount = df['discount']
price = df['price'].str.extract(r'(\d+\.?\d+)',expand=False)

#### 1.3 缺失值处理


In [ ]:
# 创建新的DataFrame
df2 = pd.DataFrame({
    'bedroom': bedroom ,
    'livingRoom' : livingRoom ,
    'storey' : storey ,
    'area' : area ,
    'orientation' : orientation ,
    'constructionTime' : constructionTime ,
    'discount' : discount ,
    'price' : price
} 
)

In [ ]:
#观察新创建的表单结构
df2.head()

In [ ]:
# 将缺少bedroom的数据删除
df2.dropna(subset=['bedroom'],inplace=True)
#用中位数填充constructionTime；用0填充未知的discount
df2.fillna({'constructionTime' : df2.constructionTime.median(),
            'discount': 0 },inplace=True)

In [ ]:
#重新观察缺失值
df2.info()

#### 1.4 数据类型转换
说明：
1. 缺失值填充一定在数据类型转换之前，因为数据类型转换时不能有缺失值；
2. 若要在缺失值填充之进行数据类型转换，可将数据保存为其他格式，重新读取数据，pandas自动进行数据类型转换。

In [ ]:
df2 = df2.astype(dtype={'bedroom':'int16',
                  'livingRoom':'int16',
                  'area':'float',
                  'constructionTime': 'int16',
                  'price':'float',
                  'discount':'str',
                  'orientation':'str'
                  })

In [ ]:
df2.info()

In [ ]:
# map函数不能单独在单元格内重复运行
df2.storey  = df2.storey.map({'底层':1,'低层':2,'中层':3,'高层':4,'顶层':5} )

In [ ]:
df2.orientation.unique()

In [ ]:
df2.orientation = df2.orientation.map({'南北向':1, '南向':2, '东向':3, '西南向':4, '北向':5, '东南向':6, '东北向':7, '西向':8, '西北向':9, '东西向':10})

In [ ]:
df2.discount.unique()

In [ ]:
df2.discount = df2.discount.map({'满五' :3 , '满二':2 , '0' :1 ,'配套成熟' :4})

In [ ]:
df2.area = (df2.area - df2.area.min())/(df2.area.max()-df2.area.min())

#### 1.5 描述性统计处理

In [ ]:
df2.describe()

In [ ]:
#定义新变量：目的：便于观察唯一值
x = df2.columns
print(x)

In [ ]:
df2[x[0]].value_counts()

In [ ]:
df2[x[1]].value_counts()

In [ ]:
#livingroom中有0值，通过索引查看原始数据，查看是否有误
#1.获取索引
y = df2.query('livingRoom == 0').index
#2.查看原始数据
df.iloc[y,:]

In [ ]:
df2[x[2]].value_counts()

In [ ]:
df2[x[4]].value_counts()

In [ ]:
df2[x[5]].unique()

In [ ]:
df2[x[6]].value_counts()

In [ ]:
#由于在discount中“配套成熟“的数据量太少，不具有代表性，因此删除
#删除方法1:生成新Dataframe
data = df2.query('discount != 4 ')

#删除方法2: 直接在原Dataframe上删除
#index1 = df2.query('discount = "配套成熟" ').index
#df2.drop(index=index1,inplace=True)

In [ ]:
data.discount.value_counts()

In [ ]:
data.columns

In [ ]:
# 观察其他类型变量与价格之间的关系：
import matplotlib.pyplot as plt
fig = plt.figure()
x = data.columns 
for i in range(0,4,1) :
    plt.subplot(2,2,i+1)
    plt.scatter(data[x[i]],data[x[7]])
    plt.title(x[i])
    plt.subplots_adjust(hspace = 0.3)
fig = plt.figure()
for i in range(4,7,1) :
    plt.subplot(2,2,i-3)
    plt.scatter(data[x[i]],data[x[7]])
    plt.title(x[i])
    plt.subplots_adjust(hspace = 0.3) 


从散点图中初步分析：
1. 价格与卧室数量：
* 价格随着卧室数量增加而增加，在卧室数量为6时达到最大，
* 在8、9之间下降，但由于卧室数量为8、9个的数据量只有3个，因此结论极有可能不准确。
2. 价格与客厅数量：
* 价格随客厅数量增加而增加，在卧室数量为2时达到最大，随后下降。
3. 价格与层高：
* 近似无关；
* 可能原因：数据划分过于粗略；
* 改进方向：可根据层数与层高中心划分；
* 例子：根据常识：层数时少时底层价格高，层数多时低层价格低。
4. 价格与面积
* 近似正相关；
5. 价格与朝向：
* 房子数量集中在南北向、南向，
* 高价格的房子也集中南北向、南向；
6. 价格与时间：
* 时间越近，房子数量越多，高价格房子也越多。
7. 价格与优惠
* 近似无关；
* 可能原因：优惠越多，说明房子有人居住时间越长，有利与不利因素相互抵消。

### 二.建立决策树模型
#### 2.1 划分训练集和测试集

In [ ]:
#由于要建立决策树二分类模型，因此需先将价格划分为二分类变量，
def priceSplit(x) :
    if x <=  250:
        x = 0
    else :
        x = 1
    return x 
data.price = data.price.map(priceSplit)
data.price.value_counts()

In [ ]:
#选出属性和标签，将其划分为训练集和测试集
from sklearn.model_selection import train_test_split
X = data.drop('price',axis=1)
Y = data.price
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.5)

#### 2.2 建立分类树

In [ ]:
#模型初步训练与评价
from sklearn import tree
model =tree.DecisionTreeClassifier()
model.fit(X_train,Y_train)
from sklearn.metrics import roc_auc_score
score1 = roc_auc_score(Y_test,model.predict_proba(X_test)[:,1])
score2 = model.score(X_test,Y_test)
print(score1,score2)

#### 2.3 参数调优

In [ ]:
# 参数调优：
from sklearn import tree
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth': range(3,20,1),'criterion':['gini','entory'],'min_samples_leaf':range(5,20,1)}
grid_search = GridSearchCV(model,parameters,scoring='roc_auc',cv=5)
grid_search.fit(X_train,Y_train)
grid_search.best_params_

In [ ]:
#参数调优后重新建模并评价模型
modelBest= tree.DecisionTreeClassifier(max_depth=3,criterion='gini',min_samples_leaf=17)
modelBest.fit(X_train,Y_train)
s1 = modelBest.score(X_train,Y_train)
s2 = modelBest.score(X_test,Y_test)
s3 = roc_auc_score(Y_test,modelBest.predict_proba(X_test)[:,1])
print(s1,s2,s3)

#### 3.1 ROC曲线

In [ ]:
#建立SVM模型
from sklearn.svm import SVC
clf = SVC(probability= True)
clf.fit(X_train,Y_train)

In [ ]:
### 将参数调优后的决策树二分类模型与SVM模型使用ROC曲线进行比较
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
fig = plt.figure()
fpr1,tpr1, thresholds1 = roc_curve(Y_test,modelBest.predict_proba(X_test)[:,1])
plt.plot(fpr1,tpr1,label='Tree',linestyle = '--')
plt.legend()
fpr2,tpr2, thresholds2 = roc_curve(Y_test,clf.predict_proba(X_test)[:,1])
plt.plot(fpr2,tpr2,label='SVM',c='red')
plt.legend()
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title(r"$Receiver\ operating\ characteristic\ example$")

In [ ]:
#两个模型评价
from sklearn.metrics import roc_auc_score
roc1 = roc_auc_score(Y_test,modelBest.predict_proba(X_test)[:,1])
roc2 = roc_auc_score(Y_test,clf.predict_proba(X_test)[:,1])
print(roc1,roc2)

### 3.2 N重交叉验证

In [ ]:
#k折交叉验证：
from sklearn.model_selection import cross_val_score
acc = cross_val_score(modelBest,X,Y,cv=5,scoring='roc_auc')
print(acc)

#### 3.3 混淆矩阵

In [ ]:
#混淆矩阵
from sklearn.metrics import confusion_matrix
Y_Pred = model.predict(X_test)
c= confusion_matrix(Y_test, Y_Pred)
print(c)

结果解读：


在参数调优后，使用训练集建立相对最优模型，使用测试集进行预测，通过观察混淆矩阵，我我们可以得到预测结果：
1. 在1442（1368+74）个负例中，预测正确的概率为：94.9%
2. 259（89+170）个正例中，预测正确的概率为：65.64%


模型还有较大改进空间。